In [2]:
import numpy as np 
import matplotlib.pyplot as plt 
import sys
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from imblearn.pipeline import Pipeline
import time
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import warnings 
from sklearn.preprocessing import RobustScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier
#from sklearn.pipeline import Pipeline
#from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [3]:
# check in the file
monopati = "/kaggle/input/sloan-digital-sky-survey-dr16/"
onomaSkyserver = "Skyserver_12_30_2019 4_49_58 PM.csv"

#create the dataframe of the csv file 
data = pd.read_csv(monopati+onomaSkyserver)


In [5]:
# checking in the dataframe 
print("The infos for the sky dataframe \n")
print(data.info())
print("\nAre there any zeros or missing values? \n")
print(data.isnull().sum())
print("\nDifferent metrics from dataset \n")
print(data.describe())
print("\nReverse different metrics from dataset \n")
print(data.describe().T)

In [4]:
# checking if dataset is imbalanced
print(data['class'].value_counts())

#with the difficult way 
# df --> numpy 
sky = pd.DataFrame(data).to_numpy()

#print(sky[:,13]) #for some reason the class -> 13
klaseis, number_in_klaseis = np.unique(sky[:,13],return_counts=True)

print("There are "+str(number_in_klaseis[0])+" samples of "+str(klaseis[0]))
print("There are "+str(number_in_klaseis[1])+" samples of "+str(klaseis[1]))
print("There are "+str(number_in_klaseis[2])+" samples of "+str(klaseis[2]))
print("The biggest different between classes is : "+str(max(number_in_klaseis)/min(number_in_klaseis)))
if max(number_in_klaseis)/min(number_in_klaseis) > 1.5 :
    print("WARNING! The dataset is imbalanced!")

In [5]:
def sns_distribution(plotter,titlos):
    sns.set_style('darkgrid')
    plt.figure(figsize = (8,6))
    plt.title(titlos)
    sns.countplot(plotter, palette = 'magma')
    plt.show()
    return None


In [16]:
#let's check our classes 
sns_distribution(data['class'],"Classes of dataset")

In [6]:
#################################
""" Make the dataset balanced """
#################################

#take X, y of dataset 
X = data.drop('class',axis=1).values
y = data['class'].values
print(X.shape)
print(y.shape)

#Oversampling 
# define oversampling strategy
oversampler = RandomOverSampler(sampling_strategy=dict({'STAR': 38096, 'GALAXY': 51323, 'QSO': 16000}))
# fit and apply the transform
X_over, y_over = oversampler.fit_resample(X, y)
print(X_over.shape)
print(y_over.shape)

#Downsampling 
# define undersampling strategy
downsampler = RandomUnderSampler(sampling_strategy=dict({'STAR': 17500, 'GALAXY': 22000, 'QSO': 16000}))
# fit and apply the transform
X_down, y_down = downsampler.fit_resample(X_over, y_over)
print(X_down.shape)
print(y_down.shape)



In [7]:
#check if dataset is balanced
klaseis_bal, number_in_klaseis_bal = np.unique(y_down,return_counts=True)

print("There are "+str(number_in_klaseis_bal[0])+" samples of "+str(klaseis_bal[0]))
print("There are "+str(number_in_klaseis_bal[1])+" samples of "+str(klaseis_bal[1]))
print("There are "+str(number_in_klaseis_bal[2])+" samples of "+str(klaseis_bal[2]))
print("The biggest different between classes is : "+str(max(number_in_klaseis_bal)/min(number_in_klaseis_bal)))

if max(number_in_klaseis_bal)/min(number_in_klaseis_bal) > 1.5 :
    print("WARNING! The dataset is imbalanced!")
else : 
    print("Teehee! The dataset is balanced")
    
sns_distribution(y_down,"Classes of Balanced dataset")

In [8]:
###########################
""" split file into X y """
###########################

X_train, X_test, y_train, y_test = train_test_split(X_down, y_down, test_size=0.30, shuffle=True)

#check if splitting is okey for y train
y_tr_class, plithos_y_tr_class = np.unique(y_train,return_counts=True)
print("There are "+str(plithos_y_tr_class[0])+" samples of "+str(y_tr_class[0]))
print("There are "+str(plithos_y_tr_class[1])+" samples of "+str(y_tr_class[1]))
print("There are "+str(plithos_y_tr_class[2])+" samples of "+str(y_tr_class[2]))
print("The biggest different between classes is : "+str(max(plithos_y_tr_class)/min(plithos_y_tr_class)))
sns_distribution(y_train,"Classes of y train")

#check if splitting is okey for y test
y_te_class, plithos_y_te_class = np.unique(y_test,return_counts=True)
print("There are "+str(plithos_y_te_class[0])+" samples of "+str(y_te_class[0]))
print("There are "+str(plithos_y_te_class[1])+" samples of "+str(y_te_class[1]))
print("There are "+str(plithos_y_te_class[2])+" samples of "+str(y_te_class[2]))
print("The biggest different between classes is : "+str(max(plithos_y_te_class)/min(plithos_y_te_class)))
sns_distribution(y_test,"Classes of y test")


## take a slice of the dataset 
## to run quickly the evaluation

# define undersampling strategy cut 0
down_0 = RandomUnderSampler(sampling_strategy=dict({'STAR': 1600, 'GALAXY': 2100, 'QSO': 1300}))
# fit and apply the transform
X_sample, y_sample = down_0.fit_resample(X_down, y_down)
#print(X_sample.shape)
#print(y_sample.shape)

Xs_train, Xs_test, ys_train, ys_test = train_test_split(X_sample, y_sample, test_size=0.30, shuffle=True)

#check if spliting is okey for the sample y train
ys_tr_class, plithos_ys_tr_class = np.unique(ys_train,return_counts=True)
print("There are "+str(plithos_ys_tr_class[0])+" samples of "+str(ys_tr_class[0]))
print("There are "+str(plithos_ys_tr_class[1])+" samples of "+str(ys_tr_class[1]))
print("There are "+str(plithos_ys_tr_class[2])+" samples of "+str(ys_tr_class[2]))
print("The biggest different between classes is : "+str(max(plithos_ys_tr_class)/min(plithos_ys_tr_class)))
sns_distribution(ys_train,"Classes of the sample of y train")

#check if spliting is okey for the sample y test
ys_te_class, plithos_ys_te_class = np.unique(ys_test,return_counts=True)
print("There are "+str(plithos_ys_te_class[0])+" samples of "+str(ys_te_class[0]))
print("There are "+str(plithos_ys_te_class[1])+" samples of "+str(ys_te_class[1]))
print("There are "+str(plithos_ys_te_class[2])+" samples of "+str(ys_te_class[2]))
print("The biggest different between classes is : "+str(max(plithos_ys_te_class)/min(plithos_ys_te_class)))
sns_distribution(ys_test,"Classes of the sample of y test")


In [9]:
def SimpleAccuracy(taxinomitis, Xi_tr, yi_tr, Xi_te, yi_te, epilogeas, anafora):
    #declare classifier
    clf = taxinomitis
    
    #check train time
    start_time1 = time.time()
    _ = clf.fit(Xi_tr, yi_tr)
    train_time = time.time() - start_time1
    
    #check test time
    start_time2 = time.time()
    yi_predi = clf.predict(Xi_te)
    test_time = time.time() - start_time2
    
    #classification report
    if anafora :
        print(classification_report(yi_te, yi_predi))
    
    if epilogeas == 'aplo' :
        score = clf.score(Xi_te, yi_te)
    if epilogeas == 'accur' :
        score = metrics.accuracy_score(yi_te, yi_predi)
    if epilogeas == 'Fena' :
        score = metrics.f1_score(yi_te, yi_predi,average='macro')
    if epilogeas == 'Recall' :
        score = metrics.recall_score(yi_te, yi_predi,average='macro')
    if epilogeas == 'Precision' :
        score = metrics.precision_score(yi_te, yi_predi,average='macro')
  
    return clf, score, train_time, test_time

In [10]:
###############################################
"""out of the box for the imbalanced dataset"""
###############################################

Xim_train, Xim_test, yim_train, yim_test = train_test_split(X, y, test_size=0.30, shuffle=True)


In [44]:
###########
### MLP ###
###########

mlp_im, _, mlp_im_train_time, mlp_im_test_time = SimpleAccuracy(MLPClassifier(), Xim_train, yim_train, Xim_test, yim_test, 'aplo', True)
_, mlp_im_simple_score, _, _ = SimpleAccuracy(MLPClassifier(), Xim_train, yim_train, Xim_test, yim_test, 'aplo', False)
_, mlp_im_accur_score, _, _ = SimpleAccuracy(MLPClassifier(), Xim_train, yim_train, Xim_test, yim_test, 'accur', False)
_, mlp_im_f1_score, _, _ = SimpleAccuracy(MLPClassifier(), Xim_train, yim_train, Xim_test, yim_test, 'Fena', False)
_, mlp_im_recall_score, _, _ = SimpleAccuracy(MLPClassifier(), Xim_train, yim_train, Xim_test, yim_test, 'Recall', False)
_, mlp_im_precision_score, _, _ = SimpleAccuracy(MLPClassifier(), Xim_train, yim_train, Xim_test, yim_test, 'Precision', False)


In [46]:
#print the results for MLP for the imbalanced dataset
print("The simple score of MLP for the IMbalanced dataset is : "+str(mlp_im_simple_score))
print("The accuracy score of MLP for the IMbalanced dataset is : "+str(mlp_im_accur_score))
print("The f1 score of MLP for the IMbalanced dataset is : "+str(mlp_im_f1_score))
print("The recall score of MLP for the IMbalanced dataset is : "+str(mlp_im_recall_score))
print("The precision score of MLP for the IMbalanced dataset is : "+str(mlp_im_precision_score))
print("The trainig time for the MLP for the IMbalanced dataset is : "+str(mlp_im_train_time))
print("The test time for the MLP for the IMbalanced dataset is : "+str(mlp_im_test_time))


In [47]:
###########
### SVC ###
###########

svc_im, _, svc_im_train_time, svc_im_test_time = SimpleAccuracy(SVC(), Xim_train, yim_train, Xim_test, yim_test, 'aplo', True)
_, svc_simple_score, _, _ = SimpleAccuracy(SVC(), Xim_train, yim_train, Xim_test, yim_test, 'aplo', False)
_, svc_accur_score, _, _ = SimpleAccuracy(SVC(), Xim_train, yim_train, Xim_test, yim_test, 'accur', False)
_, svc_f1_score, _, _ = SimpleAccuracy(SVC(), Xim_train, yim_train, Xim_test, yim_test, 'Fena', False)
_, svc_recall_score, _, _ = SimpleAccuracy(SVC(), Xim_train, yim_train, Xim_test, yim_test, 'Recall', False)
_, svc_precision_score, _, _ = SimpleAccuracy(SVC(), Xim_train, yim_train, Xim_test, yim_test, 'Precision', False)


In [50]:
#print the results for MLP for the imbalanced dataset
print("The simple score of SVC for the IMbalanced dataset is : "+str(svc_simple_score))
print("The accuracy score of SVC for the IMbalanced dataset is : "+str(svc_accur_score))
print("The f1 score of SVC for the IMbalanced dataset is : "+str(svc_f1_score))
print("The recall score of SVC for the IMbalanced dataset is : "+str(svc_recall_score))
print("The precision score of SVC for the IMbalanced dataset is : "+str(svc_precision_score))
print("The trainig time for the SVC for the IMbalanced dataset is : "+str(svc_im_train_time))
print("The test time for the SVC for the IMbalanced dataset is : "+str(svc_im_test_time))


In [54]:
#############################################
"""out of the box for the Balanced dataset"""
#############################################

#for the MLP - balanced dataset
mlp_bal, _, mlp_bal_train_time, mlp_bal_test_time = SimpleAccuracy(MLPClassifier(), X_train, y_train, X_test, y_test, 'aplo', True)
_, mlp_bal_simple_score, _, _ = SimpleAccuracy(MLPClassifier(), X_train, y_train, X_test, y_test, 'aplo', False)
_, mlp_bal_accur_score, _, _ = SimpleAccuracy(MLPClassifier(), X_train, y_train, X_test, y_test, 'accur', False)
_, mlp_bal_f1_score, _, _ = SimpleAccuracy(MLPClassifier(), X_train, y_train, X_test, y_test, 'Fena', False)
_, mlp_bal_recall_score, _, _ = SimpleAccuracy(MLPClassifier(), X_train, y_train, X_test, y_test, 'Recall', False)
_, mlp_bal_precision_score, _, _ = SimpleAccuracy(MLPClassifier(), X_train, y_train, X_test, y_test, 'Precision', False)



In [55]:
#print the results for MLP for the Balanced dataset
print("The simple score of MLP for the Balanced dataset is : "+str(mlp_bal_simple_score))
print("The accuracy score of MLP for the Balanced dataset is : "+str(mlp_bal_accur_score))
print("The f1 score of MLP for the Balanced dataset is : "+str(mlp_bal_f1_score))
print("The recall score of MLP for the Balanced dataset is : "+str(mlp_bal_recall_score))
print("The precision score of MLP for the Balanced dataset is : "+str(mlp_bal_precision_score))
print("The trainig time for the MLP for the Balanced dataset is : "+str(mlp_bal_train_time))
print("The test time for the MLP for the Balanced dataset is : "+str(mlp_bal_test_time))


In [56]:
#for the SVC - balanced dataset
svc_bal, _, svc_bal_train_time, svc_bal_test_time = SimpleAccuracy(SVC(), X_train, y_train, X_test, y_test, 'aplo', True)
_, svc_bal_simple_score, _, _ = SimpleAccuracy(SVC(), X_train, y_train, X_test, y_test, 'aplo', False)
_, svc_bal_accur_score, _, _ = SimpleAccuracy(SVC(), X_train, y_train, X_test, y_test, 'accur', False)
_, svc_bal_f1_score, _, _ = SimpleAccuracy(SVC(), X_train, y_train, X_test, y_test, 'Fena', False)
_, svc_bal_recall_score, _, _ = SimpleAccuracy(SVC(), X_train, y_train, X_test, y_test, 'Recall', False)
_, svc_bal_precision_score, _, _ = SimpleAccuracy(SVC(), X_train, y_train, X_test, y_test, 'Precision', False)


In [57]:
#print the results for MLP for the Balanced dataset
print("The simple score of SVC for the Balanced dataset is : "+str(svc_bal_simple_score))
print("The accuracy score of SVC for the Balanced dataset is : "+str(svc_bal_accur_score))
print("The f1 score of SVC for the Balanced dataset is : "+str(svc_bal_f1_score))
print("The recall score of SVC for the Balanced dataset is : "+str(svc_bal_recall_score))
print("The precision score of SVC for the Balanced dataset is : "+str(svc_bal_precision_score))
print("The trainig time for the SVC for the Balanced dataset is : "+str(svc_bal_train_time))
print("The test time for the SVC for the Balanced dataset is : "+str(svc_bal_test_time))


In [60]:
#for the Dummy - balanced dataset
dc_bal, _, dc_bal_train_time, dc_bal_test_time = SimpleAccuracy(DummyClassifier(), X_train, y_train, X_test, y_test, 'aplo', True)
_, dc_bal_simple_score, _, _ = SimpleAccuracy(DummyClassifier(), X_train, y_train, X_test, y_test, 'aplo', False)
_, dc_bal_accur_score, _, _ = SimpleAccuracy(DummyClassifier(), X_train, y_train, X_test, y_test, 'accur', False)
_, dc_bal_f1_score, _, _ = SimpleAccuracy(DummyClassifier(), X_train, y_train, X_test, y_test, 'Fena', False)
_, dc_bal_recall_score, _, _ = SimpleAccuracy(DummyClassifier(), X_train, y_train, X_test, y_test, 'Recall', False)
_, dc_bal_precision_score, _, _ = SimpleAccuracy(DummyClassifier(), X_train, y_train, X_test, y_test, 'Precision', False)


In [61]:
#print the results for Dummy for the Balanced dataset
print("The simple score of Dummy for the Balanced dataset is : "+str(dc_bal_simple_score))
print("The accuracy score of Dummy for the Balanced dataset is : "+str(dc_bal_accur_score))
print("The f1 score of Dummy for the Balanced dataset is : "+str(dc_bal_f1_score))
print("The recall score of Dummy for the Balanced dataset is : "+str(dc_bal_recall_score))
print("The precision score of Dummy for the Balanced dataset is : "+str(dc_bal_precision_score))
print("The trainig time for the Dummy for the Balanced dataset is : "+str(dc_bal_train_time))
print("The test time for the Dummy for the Balanced dataset is : "+str(dc_bal_test_time))


In [17]:
################
### PLOT BAR ###
################

def plot_bares(pinakas, onomaTA, titlos):
    polichromo3 = ['blue','orange','green']
    plt.bar(onomaTA , pinakas,color=polichromo3)
    plt.xlabel("Estimators used")
    plt.ylabel("Estimators scores ")
    plt.title(titlos)
    plt.show()
    print('\n')
    return None

def plot_AccurANDf1(bara_1, bara_2, onomata, Leganda, titlos):
    x_len = np.arange(len(onomata))
    w_bar = 0.36
    plt.bar(x_len-(w_bar/2), bara_1, width = w_bar)
    plt.bar(x_len+(w_bar/2), bara_2, width = w_bar)
    #plt.xlabel("Estimators used")
    plt.xticks(x_len, labels = onomata)
    plt.ylabel("Scores of Classifiers ")
    plt.legend(Leganda,loc='best')
    plt.title(titlos)
    plt.show()
    return None


In [81]:
# put all the results in lists
clfs3_names = ["Dummy","MLP","SVC"]
cls3_fitted = [dc_bal, mlp_bal, svc_bal]
clfs3_simple_scores = [dc_bal_simple_score, mlp_bal_simple_score, svc_bal_simple_score]
clfs3_accur_scores = [dc_bal_accur_score, mlp_bal_accur_score, svc_bal_accur_score]
clfs3_f1_scores = [dc_bal_f1_score, mlp_bal_f1_score, svc_bal_f1_score]
clfs3_recall_scores = [dc_bal_recall_score, mlp_bal_recall_score, svc_bal_recall_score]
clfs3_precision_scores = [dc_bal_precision_score, mlp_bal_precision_score, svc_bal_precision_score]
clfs3_train_time = [dc_bal_train_time, mlp_bal_train_time, svc_bal_train_time]
clfs3_test_time = [dc_bal_test_time, mlp_bal_test_time, svc_bal_test_time]

In [78]:
plot_bares(clfs3_simple_scores, clfs3_names, "Accuracy per Classifier with simple scoring ")
plot_bares(clfs3_accur_scores, clfs3_names, "Accuracy per Classifier with accuracy scoring ")
plot_bares(clfs3_f1_scores, clfs3_names, "Accuracy per Classifier with f1 macro scoring ")
plot_bares(clfs3_recall_scores, clfs3_names, "Accuracy per Classifier with recall scoring ")
plot_bares(clfs3_precision_scores, clfs3_names, "Accuracy per Classifier with precision scoring ")
plot_bares(clfs3_train_time, clfs3_names, "Train times Per Classifier")
plot_bares(clfs3_test_time, clfs3_names, "Test times Per Classifier ")

In [74]:
plot_AccurANDf1(clfs3_accur_scores, clfs3_f1_scores, clfs3_names, ['accuracy','f1_macro'], "Accuracy per Classifier")
plot_AccurANDf1(clfs3_recall_scores, clfs3_precision_scores, clfs3_names, ['Recall','Precision'], "Recall-Precision per Classifier")
plot_AccurANDf1(clfs3_train_time, clfs3_test_time, clfs3_names,['train time','test time'],"Train - Test times")

In [11]:
"""
    markdown tables 
    def from the stack overflow 
    https://stackoverflow.com/questions/13394140/generate-markdown-tables

"""

# Translation dictionaries for table alignment
left_rule = {'<': ':', '^': ':', '>': '-'}
right_rule = {'<': '-', '^': ':', '>': ':'}

def evalute_field(record, field_spec):
    """
    Evalute a field of a record using the type of the field_spec as a guide.
    """
    if type(field_spec) is int:
        return str(record[field_spec])
    elif type(field_spec) is str:
        return str(getattr(record, field_spec))
    else:
        return str(field_spec(record))

def table(file, records, fields, headings, alignment = None):
    num_columns = len(fields)
    assert len(headings) == num_columns

    # Compute the table cell data
    columns = [[] for i in range(num_columns)]
    for record in records:
        for i, field in enumerate(fields):
            columns[i].append(evalute_field(record, field))

    # Fill out any missing alignment characters.
    extended_align = alignment if alignment != None else []
    if len(extended_align) > num_columns:
        extended_align = extended_align[0:num_columns]
    elif len(extended_align) < num_columns:
        extended_align += [('^', '<')
                           for i in range[num_columns-len(extended_align)]]

    heading_align, cell_align = [x for x in zip(*extended_align)]

    field_widths = [len(max(column, key=len)) if len(column) > 0 else 0
                    for column in columns]
    heading_widths = [max(len(head), 2) for head in headings]
    column_widths = [max(x) for x in zip(field_widths, heading_widths)]

    _ = ' | '.join(['{:' + a + str(w) + '}'
                    for a, w in zip(heading_align, column_widths)])
    heading_template = '| ' + _ + ' |'
    _ = ' | '.join(['{:' + a + str(w) + '}'
                    for a, w in zip(cell_align, column_widths)])
    row_template = '| ' + _ + ' |'

    _ = ' | '.join([left_rule[a] + '-'*(w-2) + right_rule[a]
                    for a, w in zip(cell_align, column_widths)])
    ruling = '| ' + _ + ' |'

    file.write(heading_template.format(*headings).rstrip() + '\n')
    file.write(ruling.rstrip() + '\n')
    for row in zip(*columns):
        file.write(row_template.format(*row).rstrip() + '\n')

    return None 

def markdownCreator(titlos,epikefalides,dedomena):
  
  sys.stdout.write(titlos+'\n\n')

  fields = np.ndarray.tolist(np.arange(len(epikefalides)))

  align = [('^', '<'), ('^', '^'), ('^', '<'), ('^', '^'), ('^', '>'),('^','^')]

  table(sys.stdout, dedomena, fields, epikefalides, align)

  return None

In [79]:
titlos_mark1 = 'The accuracies for the classifiers with default parameters and different metrics' 
epikefalides_mark1 = ['Classifiers', 'Dummy', 'MLP', 'SVC']
dedomena_mark1 = [('Simple Score',str(clfs3_simple_scores[0]),str(clfs3_simple_scores[1]),str(clfs3_simple_scores[2])),
                  ('Accuracy Score', str(clfs3_accur_scores[0]),str(clfs3_accur_scores[1]),str(clfs3_accur_scores[2])),
                  ('f1 macro', str(clfs3_f1_scores[0]), str(clfs3_f1_scores[1]), str(clfs3_f1_scores[2])),
                  ('Recall', str(clfs3_recall_scores[0]), str(clfs3_recall_scores[1]), str(clfs3_recall_scores[2])),
                  ('Precision', str(clfs3_precision_scores[0]), str(clfs3_precision_scores[1]), str(clfs3_precision_scores[2])),
                  ('Train time', str(clfs3_train_time[0]), str(clfs3_train_time[1]), str(clfs3_train_time[2])),
                  ('Test time', str(clfs3_test_time[0]), str(clfs3_test_time[1]), str(clfs3_test_time[2]))]

markdownCreator(titlos_mark1, epikefalides_mark1, dedomena_mark1)

In [82]:
###########################
""" confusion matrixies """
###########################

for clasi in cls3_fitted:
  #plt.figure(figsize = (9,9))
  plot_confusion_matrix(clasi, X_test, y_test) 
  plt.title("The confusion matrix for the "+str(clasi))
  plt.show()
  


In [12]:
################################
""" 10 fold cross validation """
################################

def evaluate_10F(clf, Xi, yi, folds, eidos_accuracy):

    start_time = time.time()
    scores_fcv = cross_val_score(clf, Xi, yi,
                                 cv=KFold(n_splits=folds, random_state=32, shuffle=True),
                                 scoring=eidos_accuracy)

    stop_time = time.time() - start_time 
    print("The scores on each of the "+str(folds)+" splits with "+eidos_accuracy+" for "+str(clf),scores_fcv)
    
    mean_scores_fcv = np.mean(scores_fcv)
    print("The mean score with "+eidos_accuracy+" for "+str(clf)+" is "+str(mean_scores_fcv)+"\n")
    
    return mean_scores_fcv,stop_time

In [13]:
down_hyper = RandomUnderSampler(sampling_strategy=dict({'STAR': 1600, 'GALAXY': 2100, 'QSO': 1300}))
# fit and apply the transform
X_sample, y_sample = down_hyper.fit_resample(X_down, y_down)

In [14]:
#define the lists to run 10 fcv
clfs3 = [DummyClassifier(), MLPClassifier(), SVC()]
clfs3_10F_accur_scores = []
clfs3_10F_accur_times = []
clfs3_10F_f1_scores = []
clfs3_10F_f1_times = []
clfs3_10F_recall_scores = []
clfs3_10F_recall_times = []
clfs3_10F_precision_scores = []
clfs3_10F_precision_times = []

accur_metrics = ['accuracy','f1_macro','recall_macro','precision_macro']
for clasi in clfs3:
    for metriki in accur_metrics:
        score_10f_help, time_10f_help = evaluate_10F(clasi, X_sample, y_sample, 10, metriki)
        if metriki == 'accuracy':
            clfs3_10F_accur_scores.append(score_10f_help)
            clfs3_10F_accur_times.append(time_10f_help)
        if metriki == 'f1_macro':
            clfs3_10F_f1_scores.append(score_10f_help)
            clfs3_10F_f1_times.append(time_10f_help)
        if metriki == 'recall_macro':
            clfs3_10F_recall_scores.append(score_10f_help)
            clfs3_10F_recall_times.append(time_10f_help)
        if metriki == 'precision_macro':
            clfs3_10F_precision_scores.append(score_10f_help)
            clfs3_10F_precision_times.append(time_10f_help)

In [15]:
titlos_mark2 = 'The accuracies for the classifiers with default parameters and different metrics' 
epikefalides_mark2 = ['Classifiers', 'Dummy', 'MLP', 'SVC']
dedomena_mark2 = [('Accuracy Score', str(clfs3_10F_accur_scores[0]),str(clfs3_10F_accur_scores[1]),str(clfs3_10F_accur_scores[2])),
                  ('Accuracy times', str(clfs3_10F_accur_times[0]),str(clfs3_10F_accur_times[1]),str(clfs3_10F_accur_times[2])),
                  ('f1 macro', str(clfs3_10F_f1_scores[0]), str(clfs3_10F_f1_scores[1]), str(clfs3_10F_f1_scores[2])),
                  ('f1 macro times', str(clfs3_10F_f1_times[0]), str(clfs3_10F_f1_times[1]), str(clfs3_10F_f1_times[2])),
                  ('Recall', str(clfs3_10F_recall_scores[0]), str(clfs3_10F_recall_scores[1]), str(clfs3_10F_recall_scores[2])),
                  ('Recall times', str(clfs3_10F_recall_times[0]), str(clfs3_10F_recall_times[1]), str(clfs3_10F_recall_times[2])),
                  ('Precision', str(clfs3_10F_precision_scores[0]), str(clfs3_10F_precision_scores[1]), str(clfs3_10F_precision_scores[2])),
                  ('Precision times', str(clfs3_10F_precision_times[0]), str(clfs3_10F_precision_times[1]), str(clfs3_10F_precision_times[2]))]

markdownCreator(titlos_mark2, epikefalides_mark2, dedomena_mark2)

In [20]:
clfs3_names = ["Dummy","MLP","SVC"]
plot_AccurANDf1(clfs3_10F_accur_scores, clfs3_10F_f1_scores, clfs3_names, ['accuracy','f1_macro'], "Accuracy per Classifier with 10 fold cross validation")
plot_AccurANDf1(clfs3_10F_recall_scores, clfs3_10F_precision_scores, clfs3_names, ['Recall','Precision'], "Recall-Precision per Classifier with 10 fold cross validation")


In [ ]:
from sklearn.metrics import mean_squared_error
import optuna
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

def objective(trial):

    # split data
    x_balanced=X_down
    cdf= pd.DataFrame(y_down)
    class_mapping = {label:idx for idx,label in enumerate(np.unique(cdf[0]))}
    # και κάνουμε την μετατροπή
    cdf[0] = cdf[0].map(class_mapping)  
    y_new=cdf[0].values

    y_balanced=y_new
    x_train,x_test,y_train,y_test = train_test_split(x_balanced,y_balanced,test_size=0.3,shuffle=True)
    print(np.var(x_train,axis=0))
    # Sample hyper parameters
    classifier_name = trial.suggest_categorical("classifier", ["MLP",
                                                               "SVC"])
    if classifier_name=="MLP":

        # Sample hyper parameters
        # Construct the model
        threshold= trial.suggest_int('threshold', 0,6e+03,step=100)
        n_components=trial.suggest_int('n_componenets', 2, 5,step=1)
        activation=trial.suggest_categorical('activation',['tanh','relu'])
        solver=trial.suggest_categorical('solver',['sgd','adam'])
        alpha=trial.suggest_discrete_uniform('alpha' ,0.0001, 0.005,0.001)
        learning_rate=trial.suggest_categorical('learning_rate',['constant','adaptive'])
        clf=MLPClassifier(hidden_layer_sizes= (50,100,50),activation=activation,solver=solver,alpha=alpha,
                          learning_rate=learning_rate)
        pipe= Pipeline([('selector',VarianceThreshold(threshold=threshold )),('scaler',StandardScaler()),
                             ('pca',PCA(n_components=n_components)),
                             ('clf',clf)],memory='tmp')
                           
    elif classifier_name=="SVC":
        

        # Sample hyper parameters
        C = trial.suggest_loguniform('C', 1e-10, 1)
        kernel = trial.suggest_categorical('kernel',['poly','rbf','sigmoid'])
        degree = trial.suggest_int('degree',1, 50)
        gamma = trial.suggest_loguniform('gamma',0.001,10000)
        threshold= trial.suggest_int('threshold', 0 ,6e+03,step=100)
        n_components=trial.suggest_int('n_components', 2, 5,step=1)
        # Construct the model
        clf = SVC(C=C, kernel=kernel, degree=degree,gamma=gamma)
        pipe= Pipeline([('selector',VarianceThreshold(threshold=threshold )),('scaler',StandardScaler()),
                             ('pca',PCA(n_components=n_components)),
                             ('clf',clf)],memory='tmp')
    
    # Train the model
    pipe.fit(x_train,y_train)

    # Evaluate the model
    y_pred_test = pipe.predict(x_test)
    loss = mean_squared_error(y_test,y_pred_test)
    print("Test Score:",pipe.score(x_test,y_test))
    print("Train Score:",pipe.score(x_train,y_train))
    print(classification_report(y_test,y_pred_test))
    print("\n=================")
    return loss

In [ ]:
study = optuna.create_study(direction='minimize')

# start tuning for the hyper-parameters
study.optimize(objective, n_trials=50)